<a href="https://colab.research.google.com/github/hrbolek/simodes/blob/main/notebooks/examplescz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Příklad použití knihovny simodes

Pro studium kódu v tomto textu jsou třeba znalosti jazyka Python, alespoň v základní úrovni. Pro samostudium lze doporučit tento odkaz: https://naucse.python.cz/

V textu dále jsou pomocí tří příkladů demonstrovány základní funkcionality knihovny simodes, která poskytuje nástroje pro propojení simulací založených na modelech definovaných diferenciálními rovnicemi a simulací založených na událostech ([definice](https://cs.wikipedia.org/wiki/Diskr%C3%A9tn%C3%AD_simulace)).

V poslední kapitole jsou uvedeny některé implementační detaily u vybraných funkcí.

In [1]:
# instalace knihovny simodes
!pip install simodes

## Příklad A

V tomto příkladě je ukázáno, jakým způsobem lze v simulaci pracovat s modelem definovaným pomocí diferenciální rovnice.

### Importy z knihovny

V textu dále budou využity funkce ```simpleODESolver```, ```createDataSelector``` a třída ```Simulator```. Je nutné tyto prvky z knihovny simodes naimportovat, což dělá následující segment kódu.

In [2]:
import simodes
from simodes import Simulator
from simodes import simpleODESolver
from simodes import createDataSelector
#print(dir(simodes))

### Inicializace simulace

Simulační prostředí je závislé na třídě ```Simulator```. Vytvořením její instance dostáváme k dispozici veškeré metody, které jsou nezbytné pro událostní ([DES](https://en.wikipedia.org/wiki/Discrete-event_simulation)) simulaci a simulaci založené na diferenciálních rovnicích ([ODE](https://cs.wikipedia.org/wiki/Oby%C4%8Dejn%C3%A1_diferenci%C3%A1ln%C3%AD_rovnice)).

Důležitou vlastností třídy ```Simulator``` je, že uchovává veškeré informace o simulaci v jedné datové struktuře. Tuto datovou strukturu je možné získat voláním metody ```GetState()```, jak je demonstrováno níže.

In [3]:
sim = Simulator()              # inicializace simulace
currentState = sim.GetState()  # získání aktuálního stavu simulace
print(currentState)            # výpis aktuálního stavu simulace

{'odeModels': {}, 'eventList': {'events': [], 'activeEvent': None}, 'logs': []}


Datová struktura má tři prvky

- ```odeModels```
- ```eventList```
- ```logs```

Tyto prvky uchovávají jednotlivé informace takto

- ```odeModels``` informaci o stavu modelů popsaných diferenciálními rovnicemi.
```odeModels``` je v této fázi prázdný

In [4]:
print(currentState['odeModels'])

{}


- ```eventList``` informaci o plánovaných událostech. ```eventList``` obsahuje substruktury ```events``` a ```activeEvent```.


In [5]:
print(currentState['eventList'])

{'events': [], 'activeEvent': None}


- ```logs``` spravuje zprávy vzniklé při běhu simulace (tzv. logy). ```logs``` je v této fázi prázdný.

In [6]:
print(currentState['logs'])

[]


### Definice modelu

V případě, kdy se jedná o simulaci založenou na modelech popsaných diferenciálními rovnicemi, je nutné tyto modely definovat. 

Odpovídající matematickou definici modelu je možné vyjádřit rovnicí

$$\dot x = f(t,x)$$

$x$ je stavem modelu a ten je typicky vektorem, $t$ je časem simulace.

Odpovídající funkce v jazyku Python (mimochodem, Matlab využívá stejné definice) vypadá takto:

```python
def f(t, x):
    ...
    return dx
```

Zatímco na jménech parametrů nezáleží, na jejich pořadí ano. Velmi často se místo ```x``` používá ```state```.

V těle funkce Pythonu musí být spočítána derivace ```x``` a vrácena jako návratová hodnota. Jak bylo uvedeno dříve, téměř vždy se z matematického hlediska jedná o vektor. Vektor je v jazyku Python vyjádřen jako [list](https://naucse.python.cz/2018/tieto/beginners/tieto-lists/).



Pro potřeby demonstrace využití knihovny ```simodes``` použijeme model pohybu hmotného bodu v gravitačním poli Země (bez vlivu atmosféry). Pro jednoduchost budeme uvažovat jen dvě souřadnice, přičemž jedna z nich určuje dálku a druhá výšku.

Odpovídající diferenciální rovnice pro dálku má následující tvar

$$\dot s_x = v_x$$ 

$$\dot v_x = 0$$

Pohyb v ose dálka je rovnoměrný bez zrychlení ($a=0$).

Obdobně diferenciální rovnice pro výšku má následující tvar

$$\dot s_y = v_y$$ 

$$\dot v_y = -g$$

Pohyb v ose výška je ovlivněn gravitací. Protože gravitační zrychlení je orientováno opačně, než výška, má zápornou hodnotu ($g=-9.81$).

Z výše uvedeného vyplývá, že máme 4 stavové proměnné, nebo že stav je čtyřprvkový vektor.

$$\begin{pmatrix}
s_x \\
s_y \\
v_x \\
v_y \\
\end{pmatrix}$$

Uspořádání prvků stavu je důležité pro práci se stavem v průběhu simulace. Model / uspořádání prvků stavu určuje uspořádání prvků ve vektoru v průběhu simulace. Jelikož je zde zvoleno, že na prvních dvou místech jsou $s_x$ a $s_y$, budou první dva prvky vektoru určovat polohu.

Derivaci potom můžeme vyjádřit jako

$$\begin{pmatrix}
\dot s_x \\
\dot s_y \\
\dot v_x \\
\dot v_y \\
\end{pmatrix}=
\begin{pmatrix}
v_x \\
v_y \\
0 \\
g \\
\end{pmatrix}$$

Všimněte si, že pro výpočet derivace není nutné znát čas.


Výše uvedený model vyjádřený jako algoritmus výpočtu derivace stavu vyjádřený v Pythonu je uveden níže.

In [7]:
def model2D(time, state):
    sx = state[0]
    sy = state[1]
    vx = state[2]
    vy = state[3]
    return [vx, vy, 0, -9.81]

Optimální vyjádření stejného modelu je uvedeno dále

In [8]:
def model2D(time, state):
    return [state[2], state[3], 0, -9.81]

### Zanesení modelu do simulace

Je-li model definován, je možné vytvořit instanci jen pokud známe další informace. V rámci knihovny jsou, vedle modelu, požadovány:

- čas zahájení simulace
- počáteční stav modelu
- mezní čas výpočtu (velmi často jej budete uvádět $10^{300}[s]$, což je prakticky nekonečno)
- maximální krok řešení (typicky $0.0625 [s]$)

Zavedení modelu do simulace se provádí pomocí funkcí ```simpleODESolver``` a ```AttachODESolver```.

```simpleODESolver``` z modelu (```model2D```), počátečního času (```0```), počátečního stavu (```[0, 0, 10, 10]```), mezního času (```1e300```) a maximálního kroku (```0.0625.```) vytvoří strukturu ```solverA```


In [9]:
solverA = simpleODESolver(
    model2D, 0, state0=[0, 0, 10, 10], t_bound=1e300, max_step=0.0625)

```solverA``` je potom možno vložit do simulace pomocí metody ```AttachODESolver```.

In [10]:
modelIdA = sim.AttachODESolver(solverA)

```modelIdA``` je textový identifikátor, který jednoznačně identifikuje model v simulaci a lze s jeho pomocí zjišťovat stav modelu.

In [11]:
print(modelIdA)

zsxmzamfvu


Zjištění informací o modelu.

In [12]:
currentState = sim.GetState() # získání aktuálního stavu simulace
dataODEModelu = currentState['odeModels'] # všechny modely
dataModelIdA = dataODEModelu[modelIdA] # model definovaný identifikátorem
print(dataModelIdA)           # výpis informací o modelu

{'destroyed': False, 'state': {'time': 0, 'y': [0, 0, 10, 10], 'yd': [10, 10, 0, -9.81]}}


Kompletní stav modelu má dílčí prvky, ze kterých lze vyčíst např. čas, stav, či derivaci stavu. Ze stavu lze v tomto případě určit polohu a rychlost.

In [13]:
modelIdAStav = dataModelIdA['state']['y']
print(modelIdAStav)

[0, 0, 10, 10]


Protože v simulaci nebyl proveden ještě žádný krok, je stav modelu roven počátečnímu stavu. Význam jednotlivých prvků vypsaného stavu je dán diferenciální rovnicí a jejím převedením do jazyka Python. 

V tomto konkrétním případě je poloha $s=(0;0)$ a rychlost $v=(10;10)$.

V simulaci lze současně zpracovávat více ODE modelů, v tomto případě jsou to dva se stejnou diferenciální rovnicí.

In [14]:
solverB = simpleODESolver(
    model2D, 0, state0=[100, 0, -10, 10], t_bound=1e300, max_step=0.0625)
modelIdB = sim.AttachODESolver(solverB)
print(modelIdB)

egbziwfmqd


Získání informace o druhém modelu.

In [15]:
currentState = sim.GetState() # získání aktuálního stavu simulace
dataODEModelu = currentState['odeModels'] # všechny modely
dataModelIdB = dataODEModelu[modelIdB] # model definovaný identifikátorem
print(dataModelIdB)           # výpis informací o modelu

modelIdBStav = dataModelIdB['state']['y']
print(modelIdBStav) # výpis stavu modelu

{'destroyed': False, 'state': {'time': 0, 'y': [100, 0, -10, 10], 'yd': [-10, 10, 0, -9.81]}}
[100, 0, -10, 10]


Je-li třeba zjistit seznam všech modelů v simulaci lze to realizovat pomocí následujícího kódu.

In [16]:
currentState = sim.GetState() # získání aktuálního stavu simulace
dataODEModelu = currentState['odeModels'] # všechny modely

for key, value in dataODEModelu.items(): # projdi modely
    print(key) # vypiš identifikátor modelu

zsxmzamfvu
egbziwfmqd


Je-li třeba zjistit jejich stavy, je možné použít následující kód.

In [17]:
currentState = sim.GetState() # získání aktuálního stavu simulace
dataODEModelu = currentState['odeModels'] # všechny modely

for key, value in dataODEModelu.items(): # projdi modely
    stav = dataODEModelu[key]['state']['y'] # zjisti stav
    print(key, ':\t', stav) # vypiš identifikátor a stav

zsxmzamfvu :	 [0, 0, 10, 10]
egbziwfmqd :	 [100, 0, -10, 10]


### Příprava metod pro transformaci dat

V předchozí části bylo ukázáno, jak lze k datům o stavu ODE modelů přistupovat. Pro zjednodušení zde existuje pomocná funkce ```createDataSelector```. K plnému porozumění je vhodné znát tzv. lambda funkce ([zde](https://www.fzu.cz/~dominecf/porg/lekce20.html)).

V předchozí části bylo ukázáno, jak lze získat informace o modelu, je-li znám jeho identifikátor.

In [18]:
currentState = sim.GetState() # získání aktuálního stavu simulace
dataODEModelu = currentState['odeModels'] # všechny modely
dataModelIdA = dataODEModelu[modelIdA] # model definovaný identifikátorem
print(dataModelIdA)           # výpis informací o modelu

{'destroyed': False, 'state': {'time': 0, 'y': [0, 0, 10, 10], 'yd': [10, 10, 0, -9.81]}}


Velmi často z těchto informací chceme vybrat jen specifické informace. Následující funkce vybere souřadnici x modelu.



In [19]:
def x(item):
    return item['state']['y'][0]

Demonstrace funkčnosti.

In [20]:
currentState = sim.GetState() # získání aktuálního stavu simulace
dataODEModelu = currentState['odeModels'] # všechny modely
dataModelIdA = dataODEModelu[modelIdA] # model definovaný identifikátorem

print(x(dataModelIdA)) # výpis souřadnice x

0


Funkci
```python
def x(item):
    return item['state']['y'][0]
```

Lze pomocí lambda funkce napsat ve tvaru.

In [21]:
x = lambda item: item['state']['y'][0]

Důkaz funkčnosti.

In [22]:
currentState = sim.GetState() # získání aktuálního stavu simulace
dataODEModelu = currentState['odeModels'] # všechny modely
dataModelIdA = dataODEModelu[modelIdA] # model definovaný identifikátorem

print(x(dataModelIdA)) # výpis souřadnice x

0


Takových funkcí lze vytvořit celou řadu. Je vhodné je odpovídajícím způsobem pojmenovat a uspořádat do datové struktury (ano funkce může být datovým prvkem a to nejen v Pythonu, ale třeba i v [Javascriptu](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Functions/Arrow_functions)).

Níže je zavedena proměnná ```dataDescriptor``` slouží pro extrakci vybraných parametrů entit. V tomto případě se jedná o čas a první dva prvky stavu modelu, které představují souřadnice x a y.

In [23]:
dataDescriptor = {
    't': lambda item: item['state']['time'],
    'x': lambda item: item['state']['y'][0],
    'y': lambda item: item['state']['y'][1]
}

Proměnná ```dataDescriptor``` by mohla být využita na všechny modely v simulaci za předpokladu, že jejich stavová diferenciální rovnice pracuje se stavy stejné struktury (první dva prvky stavu obsahují souřadnice $x$ a $y$).

Aby to celé dobře fungovalo, je nutné ještě definovat, na které modely v simulaci chceme toto použít. Toto realizujeme pomocí proměnné ```masterMap``` definující, které modely jsou v simulaci sledovány. Názvy ```bulletA_``` a ```bulletB_``` se použijí později.

Všimněte si, že ve funkcích jsou použity uložené identifikátory ```modelIdA``` a ```modelIdB```.

In [24]:
masterMap = {
    'bulletA_': lambda item: item[modelIdA],
    'bulletB_': lambda item: item[modelIdB],
}

Proveďmě si rekapitulaci. Proměnná ```dataDescriptor``` popisuje, jaká data chceme dostat z modelů, zatímco proměnná ```masterMap``` popisujte, které modely nás zajímají.

Na základě definovaných proměnných ```masterMap``` a ```dataDescriptor``` je vytvořena, s pomocí ```createDataSelector``` funkce  ```dataSelector```, která  bude využita v průběhu simulace.

In [25]:
dataSelector = createDataSelector(masterMap, dataDescriptor)

Demonstrace využití ```dataSelector```, kdy z celých dat je vybrána je požadovaná část.

In [26]:
simData = sim.GetState() # kompletní informace o simulaci
odeModelsData = simData['odeModels']  # výběr části o odeModelech
selectedData = dataSelector(odeModelsData)  # výběr požadovaných informací
print(selectedData)

{'bulletA_t': 0, 'bulletA_x': 0, 'bulletA_y': 0, 'bulletB_t': 0, 'bulletB_x': 100, 'bulletB_y': 0}


Všimněte si, že výstup obsahuje 6 položek, přičemž první tři mají předponu ```bulletA_``` a další tři ```bulletB_```. Srovnejte tuto skutečnost s definicí proměnné ```masterMap```. U první trojice a stejně tak u druhé trojice položky končí postupně názvy ```t```, ```x``` a ```y```. Srovnejte toto s definicí proměnné ```dataDescriptor```. Tyto názvy lze měnit podle potřeb.

### Cyklus simulace

Před spuštěním cyklu simulace je inicializována proměnná ```results```, do které budou postupně vkládány výsledky simulace. 

> Pozor, níže použitý cyklus simulace může být nekonečný, je tedy potřeba definovat podmínku ukončení. V tomto konkrétním případě je simulace ukončena po 6 krocích. 

V každém kroku je z celkových dat simulace vybrána zájmová podmnožina pomocí funkce ```dataSelector``` a její výstup je vložen do pole results.
Cyklus simulace, jak je definován níže, obsahuje v proměnné ```index``` číslo kroku a ```currentResult``` informace o stavu všech modelů v simulaci.

In [27]:
results = [] # inicializace proměnné pro ukládání výsledků
for index, currentResult in enumerate(sim.Run()):
    partialResult = dataSelector(currentResult) # výběr podmnožiny informací
    results.append(partialResult) # vložení podmnožiny k výsledkům
    if index >= 5: # po definovaném počtu kroků
        break # ukonči cyklus simulace

Po ukončení cyklu simulace je možné vypsat souhrnné výsledky.

In [28]:
print(results)

[{'bulletA_t': 0, 'bulletA_x': 0, 'bulletA_y': 0, 'bulletB_t': 0, 'bulletB_x': 100, 'bulletB_y': 0}, {'bulletA_t': 0, 'bulletA_x': 0, 'bulletA_y': 0, 'bulletB_t': 0, 'bulletB_x': 100, 'bulletB_y': 0}, {'bulletA_t': 9.999000075938193e-05, 'bulletA_x': 0.0009999000075938194, 'bulletA_y': 0.0009998509674025839, 'bulletB_t': 0, 'bulletB_x': 100, 'bulletB_y': 0}, {'bulletA_t': 9.999000075938193e-05, 'bulletA_x': 0.0009999000075938194, 'bulletA_y': 0.0009998509674025839, 'bulletB_t': 0.0001731929568755909, 'bulletB_x': 99.99826807043124, 'bulletB_y': 0.0017317824393553822}, {'bulletA_t': 0.0010998900083532014, 'bulletA_x': 0.010998900083532014, 'bulletA_y': 0.01099296622039253, 'bulletB_t': 0.0001731929568755909, 'bulletB_x': 99.99826807043124, 'bulletB_y': 0.0017317824393553822}, {'bulletA_t': 0.0010998900083532014, 'bulletA_x': 0.010998900083532014, 'bulletA_y': 0.01099296622039253, 'bulletB_t': 0.0019051225256315, 'bulletB_x': 99.98094877474368, 'bulletB_y': 0.019033422598851234}]


Výsledky lze zpracovat dalšími standardními postupy, například zobrazit jako tabulku.

In [29]:
import pandas as pd # import pandas https://pandas.pydata.org/

def displayData(data): # funkce pro zobrazení dat v tabulce
    df = pd.DataFrame(data)
    display(df)

In [30]:
displayData(results) # zobraz uložené výsledky

,bulletA_t,bulletA_x,bulletA_y,bulletB_t,bulletB_x,bulletB_y
0,0.0000,0.000000,0.000000,0.000000,100.000000,0.000000
1,0.0000,0.000000,0.000000,0.000000,100.000000,0.000000
2,0.0001,0.001000,0.001000,0.000000,100.000000,0.000000
3,0.0001,0.001000,0.001000,0.000173,99.998268,0.001732
4,0.0011,0.010999,0.010993,0.000173,99.998268,0.001732
5,0.0011,0.010999,0.010993,0.001905,99.980949,0.019033


Protože se jedná o framework podporující událostní simulaci i simulaci založenou na ODE modelech, je zcela běžné, že se v tabulce objevují stejné časy. Současně je patřičné uvést, že v simulaci se "pohybuje vždy jedním objektem" a tedy časy objektů nemusí být synchronní.

## Příklad B

V tomto příkladu je ukázáno, jak lze v simulaci pracovat s událostmi.

### Importy z knihovny

In [31]:
import simodes
from simodes import Simulator
from simodes import simpleODESolver
from simodes import createDataSelector
#print(dir(simodes))

### Inicializace simulace

Simulační prostředí je závislé na třídě Simulator. Vytvořením její instance dostáváme k dispozici veškeré metody, které jsou nezbytné pro událostní (DES) simulaci a simulaci založené na diferenciálních rovnicích (ODE).

In [32]:
sim = Simulator()
currentState = sim.GetState()
print(currentState)

{'odeModels': {}, 'eventList': {'events': [], 'activeEvent': None}, 'logs': []}


### Definice událostí v simulaci

V případě, kdy je se jedná o simulaci založenou na událostech, je nutné tyto události definovat. Událost je funkcí, jejímž prvním parametrem je čas. Níže definovaná událost / funkce jen vypíše informaci.

In [33]:
def eventComeIn(time):
    print(f'V {time}s nastala událost')

### Naplánování události v simulaci

Prvním parametrem je čas, kdy k události dojde a druhým parametrem, které funkce bude v daném čase simulace vyvolána.

In [34]:
sim.AddEvent(0, eventComeIn);

### Definice událostí v simulaci II

Častějším typem události, než jaká byla demonstrována výše, je událost, na kterou navazuje další událost. Toto lze řešit naplánováním další události v rámci obsluhy aktuální události. Událost může mít více parametrů.

> V simulaci lze samozřejmě definovat více typů událostí s různou obsluhou

V příkladu je popsán systém hromadné obsluhy s frontou FIFO a jednou obslužnou linkou.

SHO s FIFO a jednou obslužnou linkou má události:

- příchod prvku do systému (```eventComeInEx```)
- pokus o zahájení obsluhy (```tryBeginService```)
- ukončení obsluhy (```eventServiceEnd```)

In [35]:
import random

queue = [] # proměnná pro frontu
def eventComeInEx(time, addEvent):
    print(f'At {time}s someone comes in') # tisk info
    queue.append(time) # uložení do fronty
    nextTime = time + random.uniform(1.5, 3) # čas dalšího příchodu
    addEvent(nextTime, eventComeInEx, addEvent=addEvent) # naplánování dalšího příchodu
    addEvent(time, tryBeginService, addEvent=addEvent) # naplánování pokusu o zahájení obsluhy
    
service = {'who': None} # proměnná pro obsazenost obsluhy
def tryBeginService(time, addEvent):
    if len(queue) > 0: # fronta není prázdná
        if service['who'] is None: # obsluha je ready
            timeIn = queue.pop() # první ve frontě
            timeOut = timeIn + random.uniform(0.5, 2.5) # čas konce obsluhy
            service['who'] = {'systemIn': timeIn, 'serviceBegin': time, 'systemOut': timeOut}
            addEvent(timeOut, eventServiceEnd, addEvent=addEvent) # naplánování konce obsluhy
            
def eventServiceEnd(time, addEvent):
    item = service['who'] # obsluhovaný prvek
    print(f'At {time}s {item} leaves the system') # tisk info
    service['who'] = None # uvolnění obsluhy
    addEvent(time, tryBeginService, addEvent=addEvent) # naplánování pokusu o zahájení obsluhy
    

### Naplánování události v simulaci

V tomto případě událost je naplánována s extra parametrem ```addEvent```. Tento parametr je předán funkci, která událost obslouží (```eventComeInEx```).

In [36]:
sim.AddEvent(0, eventComeInEx, addEvent=sim.AddEvent);

### Cyklus simulace

> Pozor, níže použitý cyklus simulace může být nekonečný, je tedy potřeba definovat podmínku ukončení. V tomto konkrétním případě je simulace ukončena nejpozději po 6 krocích. 

V událostech je použit přímé vypsání informací, proto zde při spuštění simulace nelze ovlivnit výstup.

In [37]:
for index, currentResult in enumerate(sim.Run()):
    if index >= 5:
        break

V 0s nastala událost
At 0s someone comes in
At 0.760979308325828s {'systemIn': 0, 'serviceBegin': 0, 'systemOut': 0.760979308325828} leaves the system
At 2.658489484163905s someone comes in


## Příklad C

V tomto příkladu je ukázáno jak v simulaci průběžně ukládat informace.

### Importy z knihovny

In [38]:
import simodes
from simodes import Simulator
from simodes import simpleODESolver
from simodes import createDataSelector
#print(dir(simodes))

### Inicializace simulace

Simulační prostředí je závislé na třídě Simulator. Vytvořením její instance dostáváme k dispozici veškeré metody, které jsou nezbytné pro událostní (DES) simulaci a simulaci založené na diferenciálních rovnicích (ODE).

In [39]:
sim = Simulator()
currentState = sim.GetState()
print(currentState)

{'odeModels': {}, 'eventList': {'events': [], 'activeEvent': None}, 'logs': []}


### Ukládání logů v simulaci

Při obsluze události jsou ukládány informace do logu simulace.


In [40]:
sim.AddLog('Demo')

Výpis logů lze provést následujícím způsobem.

In [41]:
fullInfo = sim.GetState()
logs = fullInfo['logs']
for item in logs:
    print(item)

{'msg': 'Demo'}


> V simulaci lze ukládat různé typy logů. Ty jsou odlišeny ukládanými parametry.

In [42]:
sim.AddLog('Demo', time=10)

### Výpis událostí

Během simulace i po jejím ukončení lze provést výpis zaznamenaných událostí

In [43]:
fullInfo = sim.GetState()
logs = fullInfo['logs']
for item in logs:
    print(item)

{'msg': 'Demo'}
{'msg': 'Demo', 'time': 10}


### Definice událostí v simulaci

Následující simulace je upraveným příkladem B, kdy informace jsou ukládány do logu místo přímého výstupu / tisku.

In [44]:
import random

queue = []
def eventComeInEx(time, addEvent, addLog):
    addLog(f'At {time}s someone comes in', time=time)
    queue.append(time)
    nextTime = time + random.uniform(1.5, 3)
    addEvent(nextTime, eventComeInEx, addEvent=addEvent, addLog=addLog)
    addEvent(time, tryBeginService, addEvent=addEvent, addLog=addLog)
    
service = {'who': None}
def tryBeginService(time, addEvent, addLog):
    if len(queue) > 0: # queue is not empty
        if service['who'] is None: # service is ready
            timeIn = queue.pop()
            timeOut = timeIn + random.uniform(0.5, 2.5)
            service['who'] = {'systemIn': timeIn, 'serviceBegin': time, 'systemOut': timeOut}
            addEvent(timeOut, eventServiceEnd, addEvent=addEvent, addLog=addLog)
            
            
def eventServiceEnd(time, addEvent, addLog):
    item = service['who']
    
    addLog(f'At {time}s item leaves the system', time=time, item=item)
    service['who'] = None
    addEvent(time, tryBeginService, addEvent=addEvent, addLog=addLog)
    

### Naplánování události v simulaci

V tomto případě událost je naplánována s extra parametrem ```addEvent``` a ```addLog```. Tyto parametry jsou předány funkci, která událost obslouží (```eventComeInEx```).

In [45]:
sim.AddEvent(0, eventComeInEx, addEvent=sim.AddEvent, addLog=sim.AddLog)

### Cyklus simulace

> Pozor, níže použitý cyklus simulace může být nekonečný, je tedy potřeba definovat podmínku ukončení. V tomto konkrétním případě je simulace ukončena nejpozději po 6 krocích. 

In [46]:
results = []
for index, currentResult in enumerate(sim.Run()):
    if index >= 5:
        break

### Výpis logů

In [47]:
state = sim.GetState()
logs = state['logs']
for item in logs:
    print(item)

{'msg': 'Demo'}
{'msg': 'Demo', 'time': 10}
{'msg': 'At 0s someone comes in', 'time': 0}
{'msg': 'At 1.054290270457574s item leaves the system', 'time': 1.054290270457574, 'item': {'systemIn': 0, 'serviceBegin': 0, 'systemOut': 1.054290270457574}}
{'msg': 'At 2.808281769800976s someone comes in', 'time': 2.808281769800976}


## Imports and Special Functions

### createDataSelector Function

Funkce ```createDataSelector``` slouží pro přípravu jednoduché transformace dat v průběhu simulace. Umožňuje z celkové informace o simulaci extrahovat jen dílčí prvky. Její použití je uvedeno v další části.

> Implementaci není nutno studovat. Funkci je možné importovat přímo z knihovny.

In [48]:
def createDataSelector(masterMaps, maps):
    def extractor(dataItem):
        result = {}
        for masterName, masterFunc in masterMaps.items():
            row = masterFunc(dataItem)
            for name, func in maps.items():
                result[masterName + name] = func(row)
        return result
    return extractor

Následující funkce transformuje data získaná v příkladech / simulacích do struktury, která je vhodná pro kreslení grafů.

In [49]:
def fromArrayOfDictsToDictOfArrays(data):
    first = data[0]
    result = {}
    for key, value in first.items():
        result[key] = []
    for row in data:
        for key, value in row.items():
            result[key].append(value)
    return result

### simpleODESolver Function

Funkce ```simpleODESolver``` je funkcí, která na základě modelu, jeho počátečního stavu generuje v daném časové intervalu stavy modelu. Funkce je koncipována jako generátor a vrací jednotlivé stavy na vyžádání. Tato konkrétní implementace je založena na metodě RungeKutta. V případě potřeby lze implementaci změnit / zpřesnit. To ovšem obvykle, vzhledem ke způsobu práce simulačního prostředí s více modely, není třeba.

> Implementaci není třeba studovat. Funkci je možné importovat přímo z knihovny.

In [50]:
import scipy.integrate as integrate # for numerical solution od differential equations

def simpleODESolver(model, t0, state0, t_bound, max_step):
    if not callable(model):
        raise ValueError('Model must be callable')

    solver = integrate.RK45(fun = model, t0 = t0, y0 = state0, t_bound = t_bound, max_step = max_step)
    currentItem = {'time': solver.t, 'y': [*state0], 'yd': [*model(t0, state0)]}
    while True:
        yield currentItem # send signal, inform about current result
        message = solver.step()
        currentItem = {'time': solver.t, 'y': [*solver.y], 'yd': [*model(solver.t, solver.y)]}
        if (not(solver.status == 'running')):
            break